In [1]:
import json
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [ ]:
spark = SparkSession.builder.getOrCreate()

    Normally IPython only displays the output of the last statement. However it can be handy to run multiple sql magics in a single cell and see the output of each execution. Setting `ast_node_interactivity` to `all` will enable that.


In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [4]:
%load_ext ipython_magic.sparksql

In [5]:
%config SparkSql.cacheTTL=3600
%config SparkSql.outputFile="/tmp/sparkdb.schema.json"

In [6]:
df = spark.read.json("contacts.json")
df.printSchema()

root
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- postalCode: string (nullable = true)
 |    |-- state: string (nullable = true)
 |    |-- streetAddress: string (nullable = true)
 |-- age: long (nullable = true)
 |-- first Name: string (nullable = true)
 |-- last Name: string (nullable = true)
 |-- phoneNumbers: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- number: string (nullable = true)
 |    |    |-- type: string (nullable = true)



In [7]:
df.createOrReplaceTempView("CONTACTS_TABLE")

In [8]:
%sparksql SHOW TABLES

database,tableName,isTemporary
,contacts_table,True


# Press tab to trigger auto completions and Ctrl-Q to format cell

In [9]:
%%sparksql
SELECT
    con.`first Name`,
    con.phoneNumbers [ 0 ].type,
    array_contains(con.phoneNumbers.type, 'home')
FROM
    contacts_table AS con


first Name,phoneNumbers[0].type,"array_contains(phoneNumbers.type AS `type`, home)"
Rack,home,True


# Create a temporary view with the --view option

In [10]:
%%sparksql --view the_exploded_table
SELECT
    *, 
    explode(con.phoneNumbers) as phoneNumber
FROM
    contacts_table AS con


create temporary view `the_exploded_table`


address,age,first Name,last Name,phoneNumbers,phoneNumber
"Row(city='San Jone', postalCode='394221', state='CA', streetAddress='126')",24,Rack,Jackon,"[Row(number='7383627627', type='home')]","Row(number='7383627627', type='home')"


In [11]:
%sparksql SHOW TABLES

database,tableName,isTemporary
,contacts_table,True
,the_exploded_table,True


# Use temporary view in subsequent queries with autocomplet suggestions

In [12]:
%%sparksql
SELECT
    *
FROM
    the_exploded_table AS the


address,age,first Name,last Name,phoneNumbers,phoneNumber
"Row(city='San Jone', postalCode='394221', state='CA', streetAddress='126')",24,Rack,Jackon,"[Row(number='7383627627', type='home')]","Row(number='7383627627', type='home')"


# Create a dataframe variable to use in pypark

In [13]:
%%sparksql --dataframe the_exploded_dataframe
SELECT
    *, 
    explode(con.phoneNumbers) as phoneNumber
FROM
    contacts_table AS con


capture dataframe to local variable `the_exploded_dataframe`


address,age,first Name,last Name,phoneNumbers,phoneNumber
"Row(city='San Jone', postalCode='394221', state='CA', streetAddress='126')",24,Rack,Jackon,"[Row(number='7383627627', type='home')]","Row(number='7383627627', type='home')"


# Continue developing your query using dataframe API

In [14]:
the_exploded_dataframe.select('phoneNumber').show()

+------------------+
|       phoneNumber|
+------------------+
|{7383627627, home}|
+------------------+



# Edit SQL within python strings
## Enjoy the same functionality as a code cell
- syntax highlighting
- code completion
- SQL formatting

In [15]:
# declare a python string
sql = '''
--start-sparksql
SELECT
    *, 
    explode(con.phoneNumbers) as phoneNumber
FROM
    contacts_table AS con
--end-sparksql
'''
print(sql)


--start-sparksql
SELECT
    *, 
    explode(con.phoneNumbers) as phoneNumber
FROM
    contacts_table AS con
--end-sparksql



In [16]:
spark.sql(sql).show()

+--------------------+---+----------+---------+--------------------+------------------+
|             address|age|first Name|last Name|        phoneNumbers|       phoneNumber|
+--------------------+---+----------+---------+--------------------+------------------+
|{San Jone, 394221...| 24|      Rack|   Jackon|[{7383627627, home}]|{7383627627, home}|
+--------------------+---+----------+---------+--------------------+------------------+



In [17]:
%%sparksql?

Docstring:
::

  %sparksql [-d DATAFRAME] [-c] [-e] [-v VIEW] [-l LIMIT] [-f OUTPUTFILE]
                [-t CACHETTL] [-a CATALOGS] [-q]
                [sql [sql ...]]

Magic that works both as %sparksql and as %%sparksql

positional arguments:
  sql                   SQL statement

optional arguments:
  -d DATAFRAME, --dataframe DATAFRAME
                        Capture dataframe in a local variable
  -c, --cache           Cache dataframe
  -e, --eager           Cache dataframe with eager load
  -v VIEW, --view VIEW  Create or replace temporary view
  -l LIMIT, --limit LIMIT
                        The maximum number of rows to display
  -f OUTPUTFILE, --outputFile OUTPUTFILE
                        Output schema to specified file, defaults to
                        /tmp/sparkdb.schema.json
  -t CACHETTL, --cacheTTL CACHETTL
                        Re-generate output schema file if older than time
                        specified (defaults to -1 seconds)
  -a CATALOGS, --catalogs 